In [1]:
import pandas as pd
from itertools import combinations

# =====================
# Dataset
# =====================
dataset = [
    ['pan','mantequilla','leche','gelatina','queso'],
    ['huevos','leche','yogurt','chocolate'],
    ['pan','queso','huevos','leche'],
    ['huevos','leche','galletas','pan'],
    ['queso','leche','galletas'],
    ['pan','huevos','cafe'],
    ['cafe','pan','yogurt'],
    ['galletas','pan','leche'],
    ['cafe','leche','queso','galletas','yogurt']
]

# =====================
# One-hot encoding
# =====================
all_items = sorted(set(i for row in dataset for i in row))
df = pd.DataFrame([{item:(item in row) for item in all_items} for row in dataset])

# =====================
# Función soporte
# =====================
def support(itemset, df):
    """Calcula el soporte de un itemset (como proporción)."""
    mask = df[list(itemset)].all(axis=1)
    return mask.mean()

# =====================
# Algoritmo Apriori
# =====================
def apriori(df, min_sup):
    n = len(df)
    items = list(df.columns)

    # 1-itemsets
    L = [{item} for item in items if support({item}, df) >= min_sup]
    freq_itemsets = [({item}, support({item}, df)) for item in items if support({item}, df) >= min_sup]

    k = 2
    while L:
        Ck = [set(a).union(b) for i, a in enumerate(L) for b in L[i+1:] if len(set(a).union(b)) == k]
        Ck = [c for c in Ck if len(c) == k]  # solo combinaciones tamaño k

        # quitar duplicados
        Ck = list(map(frozenset, Ck))
        Ck = list(set(Ck))

        L = [c for c in Ck if support(c, df) >= min_sup]

        # guardar
        freq_itemsets.extend([(set(c), support(c, df)) for c in L])
        k += 1

    return freq_itemsets

# =====================
# Reglas de asociación
# =====================
def generate_rules(freq_itemsets, df, min_conf):
    rules = []
    for itemset, sup in freq_itemsets:
        if len(itemset) < 2:
            continue
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = set(antecedent)
                consequent = itemset - antecedent
                sup_itemset = support(itemset, df)
                sup_antecedent = support(antecedent, df)
                conf = sup_itemset / sup_antecedent if sup_antecedent > 0 else 0
                if conf >= min_conf:
                    rules.append((antecedent, consequent, sup_itemset, conf))
    return rules

# =====================
# Ejecutar
# =====================
min_sup = 0.3
min_conf = 0.4

freq_itemsets = apriori(df, min_sup)
rules = generate_rules(freq_itemsets, df, min_conf)

print("✅ Conjuntos frecuentes:")
for items, sup in freq_itemsets:
    print(f"{items} -> soporte={sup:.2f}")

print("\n✅ Reglas de asociación:")
for ant, cons, sup, conf in rules:
    print(f"{ant} -> {cons}, soporte={sup:.2f}, confianza={conf:.2f}")

✅ Conjuntos frecuentes:
{'cafe'} -> soporte=0.33
{'galletas'} -> soporte=0.44
{'huevos'} -> soporte=0.44
{'leche'} -> soporte=0.78
{'pan'} -> soporte=0.67
{'queso'} -> soporte=0.44
{'yogurt'} -> soporte=0.33
{'leche', 'queso'} -> soporte=0.44
{'pan', 'leche'} -> soporte=0.44
{'pan', 'huevos'} -> soporte=0.33
{'leche', 'huevos'} -> soporte=0.33
{'galletas', 'leche'} -> soporte=0.44

✅ Reglas de asociación:
{'leche'} -> {'queso'}, soporte=0.44, confianza=0.57
{'queso'} -> {'leche'}, soporte=0.44, confianza=1.00
{'pan'} -> {'leche'}, soporte=0.44, confianza=0.67
{'leche'} -> {'pan'}, soporte=0.44, confianza=0.57
{'pan'} -> {'huevos'}, soporte=0.33, confianza=0.50
{'huevos'} -> {'pan'}, soporte=0.33, confianza=0.75
{'leche'} -> {'huevos'}, soporte=0.33, confianza=0.43
{'huevos'} -> {'leche'}, soporte=0.33, confianza=0.75
{'galletas'} -> {'leche'}, soporte=0.44, confianza=1.00
{'leche'} -> {'galletas'}, soporte=0.44, confianza=0.57


In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Transacciones
dataset = [
    ['pan','mantequilla','leche','gelatina','queso'],
    ['huevos','leche','yogurt','chocolate'],
    ['pan','queso','huevos','leche'],
    ['huevos','leche','galletas','pan'],
    ['queso','leche','galletas'],
    ['pan','huevos','cafe'],
    ['cafe','pan','yogurt'],
    ['galletas','pan','leche'],
    ['cafe','leche','queso','galletas','yogurt']
]

# One-hot encoding
df = pd.DataFrame(dataset)
all_items = sorted(set(i for row in dataset for i in row))
df_enc = pd.DataFrame([{item:(item in row) for item in all_items} for row in dataset])

# Apriori
frequent = apriori(df_enc, min_support=0.3, use_colnames=True)
rules = association_rules(frequent, metric="confidence", min_threshold=0.4)

print("Conjuntos frecuentes:")
print(frequent)
print("\nReglas de asociación:")
print(rules[['antecedents','consequents','support','confidence','lift']])

Conjuntos frecuentes:
     support           itemsets
0   0.333333             (cafe)
1   0.444444         (galletas)
2   0.444444           (huevos)
3   0.777778            (leche)
4   0.666667              (pan)
5   0.444444            (queso)
6   0.333333           (yogurt)
7   0.444444  (galletas, leche)
8   0.333333    (leche, huevos)
9   0.333333      (pan, huevos)
10  0.444444       (pan, leche)
11  0.444444     (leche, queso)

Reglas de asociación:
  antecedents consequents   support  confidence      lift
0  (galletas)     (leche)  0.444444    1.000000  1.285714
1     (leche)  (galletas)  0.444444    0.571429  1.285714
2     (leche)    (huevos)  0.333333    0.428571  0.964286
3    (huevos)     (leche)  0.333333    0.750000  0.964286
4       (pan)    (huevos)  0.333333    0.500000  1.125000
5    (huevos)       (pan)  0.333333    0.750000  1.125000
6       (pan)     (leche)  0.444444    0.666667  0.857143
7     (leche)       (pan)  0.444444    0.571429  0.857143
8     (leche)    